# W04 Cassandra Python Tutorial

### Import packages

In [1]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [2]:
clstr = Cluster()
session = clstr.connect()

### Use session to 'talk' to cassandra

In [3]:
session.execute("drop keyspace m14;")

In [4]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


In [5]:
session.execute("CREATE KEYSPACE IF NOT EXISTS m14 WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':2}")

In [6]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

m14
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


Create a inventory table for testing

In [7]:
session.execute("""
CREATE TABLE IF NOT EXISTS m14.inventory ( 
    sku INT, 
    name TEXT, 
    description TEXT, 
    warehouse_num int, 
    PRIMARY KEY(sku)
);
""")

### Load data

In [8]:
df = pd.read_csv('../W04/m14.csv')
df

,sku,name,description,warehouse_num
0,1000,'Biscuit','parle-g',95
1,1001,'Chocolate','nestle',97
2,1002,'Bread','nature own',87
3,1003,'milk','visaka',92
4,1004,'eggs','heritage',65
5,1005,'coffee','Nescafe',92
6,1006,'Sugar','tata',87


In [9]:
for index, row in df.iterrows():
    print(f"sku = {row.sku}, name ={row.name}, description = {row.description}, warehouse_num = {row.warehouse_num}")

sku = 1000, name =0, description = 'parle-g', warehouse_num = 95
sku = 1001, name =1, description = 'nestle', warehouse_num = 97
sku = 1002, name =2, description = 'nature own', warehouse_num = 87
sku = 1003, name =3, description = 'visaka', warehouse_num = 92
sku = 1004, name =4, description = 'heritage', warehouse_num = 65
sku = 1005, name =5, description = 'Nescafe', warehouse_num = 92
sku = 1006, name =6, description = 'tata', warehouse_num = 87


In [10]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO m14.inventory (sku,name,description,warehouse_num)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]});
        """
       )
    session.execute(f"""
        INSERT INTO m14.inventory (sku,name,description,warehouse_num)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]});
        """
       )


        INSERT INTO m14.inventory (sku,name,description,warehouse_num)     
        VALUES (1000, 'Biscuit', 'parle-g', 95);
        

        INSERT INTO m14.inventory (sku,name,description,warehouse_num)     
        VALUES (1001, 'Chocolate', 'nestle', 97);
        

        INSERT INTO m14.inventory (sku,name,description,warehouse_num)     
        VALUES (1002, 'Bread', 'nature own', 87);
        

        INSERT INTO m14.inventory (sku,name,description,warehouse_num)     
        VALUES (1003, 'milk', 'visaka', 92);
        

        INSERT INTO m14.inventory (sku,name,description,warehouse_num)     
        VALUES (1004, 'eggs', 'heritage', 65);
        

        INSERT INTO m14.inventory (sku,name,description,warehouse_num)     
        VALUES (1005, 'coffee', 'Nescafe', 92);
        

        INSERT INTO m14.inventory (sku,name,description,warehouse_num)     
        VALUES (1006, 'Sugar', 'tata', 87);
        


In [11]:
session.execute("create index w_num on m14.inventory(warehouse_num);")

In [13]:
rows = session.execute("select (sku,name,description,warehouse_num) from m14.inventory")
for row in rows:
    print(f"sku={row[0][0]}, name={row[0][1]}, description={row[0][2]}, warehouse_num={row[0][3]}")

sku=1006, name=Sugar, description=tata, warehouse_num=87
sku=1004, name=eggs, description=heritage, warehouse_num=65
sku=1005, name=coffee, description=Nescafe, warehouse_num=92
sku=1001, name=Chocolate, description=nestle, warehouse_num=97
sku=1003, name=milk, description=visaka, warehouse_num=92
sku=1002, name=Bread, description=nature own, warehouse_num=87
sku=1000, name=Biscuit, description=parle-g, warehouse_num=95


In [12]:
rows = session.execute("select sku,name,description from m14.inventory where warehouse_num = 92 ALLOW FILTERING;")
for row in rows:
    print(f"{row[0]}, {row[1]}, {row[2]}")

1005, coffee, Nescafe
1003, milk, visaka
